<a href="https://colab.research.google.com/github/Shadman-spath004/CS895_Assignment_2_Shadman01243104/blob/main/CS895_Assignment2_digitset_Shadman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing key libraries, and reading data
import pandas as pd
import numpy as np

np.random.seed(2409)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Digit MNIST/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Digit MNIST/test.csv')

In [ ]:
df_train.head() # 784 features, 1 label

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Splitting into training and validation dataset
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.values.reshape(33600, 784) #(33600, 784)
X_cv = X_cv.values.reshape(8400, 784) #(8400, 784)

X_test = X_test.values.reshape(28000, 784)

In [ ]:
#Data cleaning, normalization and selection
print((min(X_train[1]), max(X_train[1])))

(0, 255)


In [ ]:
#As the pixel intensities are currently between the range of 0 and 255, we proceed to normalize the features, using broadcasting. In addition, we proceed to convert our labels from a class vector to binary One Hot Encoded
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [ ]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
#Model Fitting
#We proceed by fitting several simple neural network models using Keras (with TensorFlow as our backend) and collect their accuracy. The model that performs the best on the validation set will be used as the model of choice for the competition.

#Model 1: Simple Neural Network with 2 layers (300, 200)

#In our first model, we will use the Keras library to train a neural network with the activation function set as ReLu. To determine which class to output, we will rely on the SoftMax function

In [ ]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 200
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '4' layers - input layer, 2 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 200)               60200     
                                                                 
 Output_Layer (Dense)        (None, 10)                2010      
                                                                 
Total params: 297,710
Trainable params: 297,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 5s - loss: 1.3664 - accuracy: 0.6854 - val_loss: 0.6939 - val_accuracy: 0.8501 - 5s/epoch - 15ms/step
Epoch 2/20
336/336 - 3s - loss: 0.5458 - accuracy: 0.8660 - val_loss: 0.4481 - val_accuracy: 0.8848 - 3s/epoch - 10ms/step
Epoch 3/20
336/336 - 3s - loss: 0.4128 - accuracy: 0.8889 - val_loss: 0.3740 - val_accuracy: 0.8960 - 3s/epoch - 10ms/step
Epoch 4/20
336/336 - 3s - loss: 0.3598 - accuracy: 0.9000 - val_loss: 0.3373 - val_accuracy: 0.9057 - 3s/epoch - 10ms/step
Epoch 5/20
336/336 - 3s - loss: 0.3289 - accuracy: 0.9076 - val_loss: 0.3146 - val_accuracy: 0.9098 - 3s/epoch - 8ms/step
Epoch 6/20
336/336 - 2s - loss: 0.3064 - accuracy: 0.9134 - val_loss: 0.2984 - val_accuracy: 0.9165 - 2s/epoch - 6ms/step
Epoch 7/20
336/336 - 2s - loss: 0.2892 - accuracy: 0.9180 - val_loss: 0.2849 - val_accuracy: 0.9183 - 2s/epoch - 6ms/step
Epoch 8/20
336/336 - 2s - loss: 0.2752 - accuracy: 0.9221 - val_loss: 0.2754 - val_accuracy: 0.9211 - 2s/epoch - 6ms/step
Epoch 9/20
336/336 -

In [ ]:
#Using a 2 layer neural network with:

#20 training epochs
#A training batch size of 100
#Hidden layers set as (300, 100, 100, 200)
#Learning rate of 0.1
#Achieved a training score of around 94% and a test score of around 93%.

#Can we do better if we were to change the optimizer? To find out, we use the Adam optimizer for our second model, while maintaining the same parameter values for all other parameters.

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 3s - loss: 0.3139 - accuracy: 0.9094 - val_loss: 0.1585 - val_accuracy: 0.9539 - 3s/epoch - 8ms/step
Epoch 2/20
336/336 - 2s - loss: 0.1203 - accuracy: 0.9647 - val_loss: 0.1145 - val_accuracy: 0.9640 - 2s/epoch - 7ms/step
Epoch 3/20
336/336 - 2s - loss: 0.0760 - accuracy: 0.9769 - val_loss: 0.0971 - val_accuracy: 0.9712 - 2s/epoch - 6ms/step
Epoch 4/20
336/336 - 2s - loss: 0.0527 - accuracy: 0.9837 - val_loss: 0.1025 - val_accuracy: 0.9694 - 2s/epoch - 7ms/step
Epoch 5/20
336/336 - 2s - loss: 0.0366 - accuracy: 0.9885 - val_loss: 0.0965 - val_accuracy: 0.9730 - 2s/epoch - 6ms/step
Epoch 6/20
336/336 - 2s - loss: 0.0274 - accuracy: 0.9917 - val_loss: 0.1047 - val_accuracy: 0.9706 - 2s/epoch - 7ms/step
Epoch 7/20
336/336 - 2s - loss: 0.0247 - accuracy: 0.9921 - val_loss: 0.1061 - val_accuracy: 0.9719 - 2s/epoch - 7ms/step
Epoch 8/20
336/336 - 2s - loss: 0.0150 - accuracy: 0.9949 - val_loss: 0.1093 - val_accuracy: 0.9726 - 2s/epoch - 7ms/step
Epoch 9/20
336/336 - 2s 

In [ ]:
#As it turns out, it does appear to be the case that the optimizer plays a crucial part in the validation score. 
#In particular, the model which relies on 'Adam' as its optimizer tend to perform 1.5 - 2.5% better on average. 
#Going forward, we will use 'Adam' as our optimizer of choice.

#What if we changed the learning rate from 0.1 to 0.01, or 0.5? Will it have any impact on the accuracy? Model 2A

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 3s - loss: 0.3137 - accuracy: 0.9097 - val_loss: 0.1600 - val_accuracy: 0.9545 - 3s/epoch - 9ms/step
Epoch 2/20
336/336 - 2s - loss: 0.1205 - accuracy: 0.9630 - val_loss: 0.1033 - val_accuracy: 0.9688 - 2s/epoch - 6ms/step
Epoch 3/20
336/336 - 2s - loss: 0.0763 - accuracy: 0.9770 - val_loss: 0.1031 - val_accuracy: 0.9667 - 2s/epoch - 6ms/step
Epoch 4/20
336/336 - 2s - loss: 0.0553 - accuracy: 0.9829 - val_loss: 0.0881 - val_accuracy: 0.9720 - 2s/epoch - 7ms/step
Epoch 5/20
336/336 - 2s - loss: 0.0404 - accuracy: 0.9874 - val_loss: 0.0971 - val_accuracy: 0.9717 - 2s/epoch - 6ms/step
Epoch 6/20
336/336 - 2s - loss: 0.0266 - accuracy: 0.9917 - val_loss: 0.0843 - val_accuracy: 0.9760 - 2s/epoch - 7ms/step
Epoch 7/20
336/336 - 2s - loss: 0.0214 - accuracy: 0.9934 - val_loss: 0.0814 - val_accuracy: 0.9757 - 2s/epoch - 7ms/step
Epoch 8/20
336/336 - 3s - loss: 0.0186 - accuracy: 0.9939 - val_loss: 0.1060 - val_accuracy: 0.9718 - 3s/epoch - 8ms/step
Epoch 9/20
336/336 - 3s 

In [ ]:
#Model 2B

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 3s 8ms/step - loss: 0.3201 - accuracy: 0.9078 - val_loss: 0.1494 - val_accuracy: 0.9570
Epoch 2/20
336/336 [==============================] - 3s 8ms/step - loss: 0.1177 - accuracy: 0.9641 - val_loss: 0.1185 - val_accuracy: 0.9655
Epoch 3/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0757 - accuracy: 0.9766 - val_loss: 0.0986 - val_accuracy: 0.9696
Epoch 4/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0547 - accuracy: 0.9822 - val_loss: 0.0932 - val_accuracy: 0.9711
Epoch 5/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0368 - accuracy: 0.9885 - val_loss: 0.0914 - val_accuracy: 0.9739
Epoch 6/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0292 - accuracy: 0.9911 - val_loss: 0.0941 - val_accuracy: 0.9732
Epoch 7/20
336/336 [==============================] - 3s 8ms/step - loss: 0.0211 - accuracy: 0.9934 - val_loss: 0.0979 - val_accuracy: 0.9733
Epoch 

In [ ]:
#The accuracy, as measured by the 3 different learning rates 0.01, 0.1 and 0.5 are around 98%, 97% and 98% respectively.
#As there are no considerable gains by changing the learning rates, we stick with the default learning rate of 0.01.

#We proceed to fit a neural network with 5 hidden layers with the features in the hidden layer set as (300, 100, 100, 100, 200) respectively. 
#To ensure that the two models are comparable, we will set the training epochs as 20, and the training batch size as 100.

In [ ]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_5 (Dense)      (None, 200)               20200     
                                                                 
 Output_Layer (Dense)        (None, 10)                201

In [ ]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 4s 10ms/step - loss: 0.3680 - accuracy: 0.8857 - val_loss: 0.1636 - val_accuracy: 0.9513
Epoch 2/20
336/336 [==============================] - 3s 9ms/step - loss: 0.1328 - accuracy: 0.9603 - val_loss: 0.1153 - val_accuracy: 0.9652
Epoch 3/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0865 - accuracy: 0.9732 - val_loss: 0.1194 - val_accuracy: 0.9649
Epoch 4/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0680 - accuracy: 0.9780 - val_loss: 0.1110 - val_accuracy: 0.9705
Epoch 5/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0496 - accuracy: 0.9845 - val_loss: 0.0978 - val_accuracy: 0.9713
Epoch 6/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0413 - accuracy: 0.9874 - val_loss: 0.0929 - val_accuracy: 0.9754
Epoch 7/20
336/336 [==============================] - 3s 9ms/step - loss: 0.0350 - accuracy: 0.9887 - val_loss: 0.1003 - val_accuracy: 0.9742
Epoch

In [ ]:
#Compared to our first model, adding an additional layer did not significantly improve the accuracy from our previous model. 
#However, there are computational costs (in terms of complexity) in implementing an additional layer in our neural network. 
#Given that the benefits of an additional layer are low while the costs are high, we will stick with the 4 layer neural network.

#We now proceed to include dropout (dropout rate of 0.3) in our second model to prevent overfitting.

In [ ]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
num_digits = 10

In [ ]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [ ]:
# Our model would have '4' layers - input layer, 2 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Output_Layer (Dense)        (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 3s 8ms/step - loss: 0.3893 - accuracy: 0.8824 - val_loss: 0.1620 - val_accuracy: 0.9535
Epoch 2/20
336/336 [==============================] - 3s 8ms/step - loss: 0.1628 - accuracy: 0.9498 - val_loss: 0.1188 - val_accuracy: 0.9665
Epoch 3/20
336/336 [==============================] - 3s 8ms/step - loss: 0.1168 - accuracy: 0.9636 - val_loss: 0.1023 - val_accuracy: 0.9690
Epoch 4/20
336/336 [==============================] - 2s 7ms/step - loss: 0.0945 - accuracy: 0.9702 - val_loss: 0.0884 - val_accuracy: 0.9736
Epoch 5/20
336/336 [==============================] - 2s 7ms/step - loss: 0.0748 - accuracy: 0.9759 - val_loss: 0.0931 - val_accuracy: 0.9739
Epoch 6/20
336/336 [==============================] - 2s 7ms/step - loss: 0.0669 - accuracy: 0.9788 - val_loss: 0.0787 - val_accuracy: 0.9760
Epoch 7/20
336/336 [==============================] - 2s 7ms/step - loss: 0.0559 - accuracy: 0.9819 - val_loss: 0.0909 - val_accuracy: 0.9749
Epoch 

In [ ]:
#With a validation score of close to 99%, we proceed to use this model to predict for the test set.

In [ ]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

140/140 [==============================] - 1s 4ms/step


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [ ]:
test_pred.to_csv('mnist_submission.csv', index = False)

In [ ]:
#Using this model, we are able to achieve a score of 0.976.